In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
from tqdm import tqdm
from os.path import join
from sebm.data import Load_Grass
from torchvision import datasets, transforms

In [13]:
data_dir
i = 1
for file in os.listdir('/Users/hao/Downloads/Imagenet-Grass/'):
    path = os.path.join('/Users/hao/Downloads/Imagenet-Grass', file)
    if file.endswith(".JPG") or file.endswith(".JPEG") or file.endswith(".jpg"):
        try:
            a = Image.open(path)
            a.save(os.path.join('/Users/hao/Downloads/Imagenet-Grass', '{:04d}.JPG'.format(i)))
            i += 1
            os.remove(path)
        except:
            os.remove(path)
    else:
        os.remove(path)

In [ ]:
def Trim_Grass(root):
    """
    Download and preprocess Grass if if doesn't exist
    """
    data_path = join(root, 'Grass')
    raw_folder = join(data_path, 'raw')
    processed_folder = join(data_path, 'processed')
    processed_path = join(processed_folder, 'data.pt')
    if os.path.exists(processed_path):
        return torch.load(processed_path)
    else:
        os.makedirs(join(raw_folder, 'jpg'), exist_ok=True)
        os.makedirs(join(processed_folder, 'jpg'), exist_ok=True)
        print('Processing...')
        data = []
        for f in tqdm(range(892)):
            filename = '%04d.JPG' % (f+1)
            file_path = join(raw_folder, 'jpg', filename)
            img = Image.open(file_path)
            img = img.convert(mode='L')
            img = transforms.ToTensor()(img)[0]
            # trim off the paddings  at left and right sides
            if img[:, 0].mean() <= 1e-2:
                if img[:, 4].mean() <= 1e-2:
                    if img[:, 9].mean() <= 1e-2:
                        if img[:, 14].mean() <= 1e-2:
                            if img[:,19].mean() <= 1e-2:
                                if img[:, 29].mean() <= 1e-2:
                                    print('30px left, image {}'.format(filename))
                                    img = img[:, 35:-35]
                                else:
                                    print('20px left, image {}'.format(filename))
                                    img = img[:, 25:-25]
                            else:
                                print('15px left, image {}'.format(filename))
                                img = img[:, 20:-20]
                        else:
                            print('10px left, image {}'.format(filename))
                            img = img[:, 15:-15]
                    else:
                        print('5px left, image {}'.format(filename))
                        img = img[:, 10:-10]
                else:
                    print('1px left, image {}'.format(filename))
                    img = img[:, 5:-5]
                    
            # trim off the paddings at the top and bottom
            if img[0, :].mean() <= 1e-2:
                if img[4, :].mean() <= 1e-2:
                    if img[9, :].mean() <= 1e-2:
                        if img[14, :].mean() <= 1e-2:
                            if img[19, :].mean() <= 1e-2:
                                if img[29, :].mean() <= 1e-2:
                                    if img[44, :].mean() <= 1e-2:
                                        print('45px top, image {}'.format(filename))
                                        img = img[50:-50, :]
                                    else:
                                        print('30px top, image {}'.format(filename))
                                        img = img[35:-35, :]
                                else:
                                    print('20px top, image {}'.format(filename))
                                    img = img[25:-25, :]
                            else:
                                print('15px top, image {}'.format(filename))
                                img = img[20:-20, :]
                        else:
                            print('10px top, image {}'.format(filename))
                            img = img[15:-15, :]
                    else:
                        print('5px top, image {}'.format(filename))
                        img = img[10:-10, :] 
                else:
                    print('1px top, image {}'.format(filename))
                    img = img[5:-5, :] 
            trimmed_img = Image.fromarray((img.numpy()*255).astype(np.uint8), mode='L')
            h, w = trimmed_img.size
            if h < 100 or w < 100:
                print('Size of image {}={}, {}'.format(filename, h, w))
            pre_transforms = transforms.Compose([
                                transforms.CenterCrop((min(h, w), min(h, w))),
                                transforms.Resize((100,100)),
                                transforms.ToTensor()])
            resized_cropped_img = Image.fromarray((pre_transforms(trimmed_img)[0].numpy()*255).astype(np.uint8), mode='L')
            resized_cropped_img.save(join(processed_folder, 'jpg', filename))
#         return img
    
img = Load_Grass('../data/')